# Event Plots (inline version)

A NB template to ilustrate how to examine the features of events

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import time
import tables as tb
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from invisible_cities.database import load_db

In [ ]:
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.tbl_functions as tbl
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [ ]:
from gui_functions import plot_pmt_waveforms, plot_pmt_signals_vs_time_mus, plot_signal_vs_time_mus,\
                           plot_s12

In [ ]:
from event_pmaps import EventPmaps, print_s12, print_s2si

### Run Parameters

In [ ]:
run_number = 3389

In [ ]:
DataPMT = load_db.DataPMT(run_number)

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],
                        'LSC/wvfms/{}/dst_waves.gdcsnext.000_{}.root.h5'.format(run_number, run_number))


In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

In [ ]:
s1par  = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=5, lmax=20, stride=4, rebin=False)
s2par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=0.5 * units.pes,  thr_s2=1 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

In [ ]:
epm = EventPmaps(run_number,
                 s1par, s2par, thr,
                 verbose=True)

In [ ]:
event=0
epm.calibrated_pmt_and_csum(event, pmtrwf)

In [ ]:
plot_pmt_signals_vs_time_mus(epm.CWF,
                                 epm.P.pmt_active,
                                 t_min      =    0,
                                 t_max      = 1300,
                                 signal_min =    -5,
                                 signal_max =  250)

There is a large S1 at around 400 mus. This is too large for the S2 and may be a hint of the issues we are facing.

In [ ]:
plot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                             epm.P.pmt_active,
                             t_min      =    0,
                             t_max      = 1300,
                             signal_min =    -5,
                             signal_max =  10)

In [ ]:
plot_signal_vs_time_mus(epm.csum, 
                            t_min=0, 
                            t_max=1300, 
                            signal_min=-5, 
                            signal_max=60)

The height of "S1" is totally inconsistent with S2. What we have here is an event with an S1 of unknown origin and an S2 probably without S1. 

In [ ]:
epm.find_s1()

An example of non-sense S1, at 60 pes. 

In [ ]:
epm.find_s2()

S2 has 7,000 pes, could be consistent with Krypton, but possibly is not (Krypton is probably around 6300 pes). 

In [ ]:
plot_s12(epm.S1)

In [ ]:
epm.s1f

In [ ]:
plot_s12(epm.S2)

In [ ]:
epm.s2f

In [ ]:
epm.find_ns1()

In [ ]:
dt = epm.s2f[0].tpeak - epm.s1f[0].tpeak

In [ ]:
dt/units.mus

In [ ]:
event=1
epm.calibrated_pmt_and_csum(event, pmtrwf)

In [ ]:
plot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                             epm.P.pmt_active,
                             t_min      =    0,
                             t_max      = 1300,
                             signal_min =    -5,
                             signal_max =  7)

In [ ]:
plot_signal_vs_time_mus(epm.csum, 
                            t_min=0, 
                            t_max=1300, 
                            signal_min=-5, 
                            signal_max=60)

In [ ]:
epm.find_s1()
epm.find_s2()
epm.find_ns1()

In [ ]:
plot_s12(epm.S1)

In [ ]:
plot_s12(epm.S2)

In [ ]:
epm.s1f

This looks like a background event in conjunction with electron rain

In [ ]:
epm.s2f

In [ ]:
event=2
epm.calibrated_pmt_and_csum(event, pmtrwf)
epm.find_s1()
epm.find_s2()
epm.find_ns1()

In [ ]:
plot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                             epm.P.pmt_active,
                             t_min      =    0,
                             t_max      = 1300,
                             signal_min =    -5,
                             signal_max =  10)

In [ ]:
plot_signal_vs_time_mus(epm.csum, 
                            t_min=0, 
                            t_max=1300, 
                            signal_min=-5, 
                            signal_max=60)

In [ ]:
plot_s12(epm.S1)

In [ ]:
plot_s12(epm.S2)

In [ ]:
epm.s1f

In [ ]:
epm.s2f

In [ ]:
def pmp(epm, event):
    epm.calibrated_pmt_and_csum(event, pmtrwf)
    
    plot_signal_vs_time_mus(epm.csum, 
                            t_min=0, 
                            t_max=1300, 
                            signal_min=-5, 
                            signal_max=60, figsize=(4,4))

    plt.show()
    print('+++S2+++++')
    epm.find_s2()
    if len(epm.S2) != 1:
        print('S2 not = 1')
        return 0
    plot_s12(epm.S2, figsize=(4,4))
    plt.show()
    print('s2f = {} '.format(epm.s2f[0]))
    
    print('+++S1+++++')
    epm.find_s1()
    if len (epm.S1) == 0:
        print('S1 == 0')
        return 0
    
    plot_s12(epm.S1, figsize=(4,4))
    
    for peak in epm.s1f:
        print('s1f = {} '.format(epm.s1f[peak]))
        dt = epm.s2f[0].tpeak - epm.s1f[peak].tpeak
        print('dt = {} mus'.format(dt/units.mus))
    

In [ ]:
def pmp_plot(epmx):
    plot_pmt_signals_vs_time_mus(epmx.CAL_PMT,
                             epmx.P.pmt_active,
                             t_min      =    0,
                             t_max      = 1300,
                             signal_min =    -5,
                             signal_max =  10, figsize=(4,4))
    plt.show()
    plot_signal_vs_time_mus(epmx.csum, 
                            t_min=0, 
                            t_max=1300, 
                            signal_min=-5, 
                            signal_max=60,figsize=(4,4))
    plt.show()
    plot_s12(epmx.S1)
    plt.show(
    plot_s12(epmx.S2))
    plt.show()

In [ ]:
pmp(epm,event=3)

S1 has large er which may be distinctive of ER

In [ ]:
pmp(epm,event=4)

In [ ]:
pmp(epm,event=5)

In [ ]:
pmp(epm,event=6)

In [ ]:
pmp(epm,event=8)

In [ ]:
pmp(epm,event=9)